# **SQLite**

**SQLite is a open source C-language library that implements a small, self-contained SQL database engine. A complete SQL database with multiple tables, indices, triggers, and views, is contained in a single disk file. With all features enabled, the library size can be less than 768 KB, depending on the target platform and compiler optimization settings.**

**Because SQLite is self-contained it can be run on Google Colab. Other database management systems like MongoDB and Postgres operate using client server architecture, which requires access to ports. However, Google Colab's environment is managed and restricts access to ports.**

<sup>Source: [SQLite Documentation](https://www.sqlite.org/) from sqlite.org</sup>

<sup>Source: [SQLite Is Public Domain](https://www.sqlite.org/copyright.html) from sqlite.org</sup>

# **Utilizing SQLite in Google Colab**

**In this tutorial we will review how to create a SQLite database, read in data into the database and run queries on the database.**

In [ ]:
#download data
!wget https://archive.ics.uci.edu/static/public/235/individual+household+electric+power+consumption.zip
!unzip /content/individual+household+electric+power+consumption.zip

In [ ]:
#check data
!head /content/household_power_consumption.txt

In [ ]:
import sqlite3
import pandas as pd
import csv

## **Reading in Data to SQLite from a pandas DataFrame**

In [ ]:
raw_data_path = '/content/household_power_consumption.txt'

In [ ]:
#read the data into a pandas DataFrame
df = pd.read_csv(raw_data_path, sep=';', low_memory=False)
print(f'The dataset has {df.shape[1]} columns and {df.shape[0]:,} rows')

In [ ]:
df.head()

In [ ]:
#create SQLite database
conn1 = sqlite3.connect('household_power.db')

In [ ]:
#write DataFrame to SQLite table
df.to_sql('household_power', conn1, if_exists='replace', index=False, dtype={
    'Date': 'TEXT',
    'Time': 'TEXT',
    'Global_active_power': 'REAL',
    'Global_reactive_power': 'REAL',
    'Voltage': 'REAL',
    'Global_intensity': 'REAL',
    'Sub_metering_1': 'REAL',
    'Sub_metering_2': 'REAL',
    'Sub_metering_3': 'REAL'
})

## **Running Queries on the SQLite Database using `pandas`**

In [ ]:
#query database to retrieve column names
query = "PRAGMA table_info(household_power)"
result = pd.read_sql(query, conn1)
print(result)

In [ ]:
#retrieve shape of data
query = """
SELECT 'rows' AS dimension, COUNT(*) AS count FROM household_power
UNION ALL
SELECT 'columns' AS dimension, COUNT(*) AS count FROM pragma_table_info('household_power');
"""
result = pd.read_sql(query, conn1)
print(result)

In [ ]:
query = "SELECT AVG(global_active_power) AS average_power FROM household_power;"
result = pd.read_sql(query, conn1)
print(result)

In [ ]:
#check for null values across all the columns
query = """
SELECT
    SUM(CASE WHEN Date IS NULL THEN 1 ELSE 0 END) AS null_date,
    SUM(CASE WHEN Time IS NULL THEN 1 ELSE 0 END) AS null_time,
    SUM(CASE WHEN global_active_power IS NULL THEN 1 ELSE 0 END) AS null_global_active_power,
    SUM(CASE WHEN global_reactive_power IS NULL THEN 1 ELSE 0 END) AS null_global_reactive_power,
    SUM(CASE WHEN Voltage IS NULL THEN 1 ELSE 0 END) AS null_voltage,
    SUM(CASE WHEN global_intensity IS NULL THEN 1 ELSE 0 END) AS null_global_intensity,
    SUM(CASE WHEN sub_metering_1 IS NULL THEN 1 ELSE 0 END) AS null_sub_metering_1,
    SUM(CASE WHEN sub_metering_2 IS NULL THEN 1 ELSE 0 END) AS null_sub_metering_2,
    SUM(CASE WHEN sub_metering_3 IS NULL THEN 1 ELSE 0 END) AS null_sub_metering_3
FROM household_power;
"""
result = pd.read_sql(query, conn1)
print(result)

In [ ]:
#close connection after finishing queries
conn1.close()

## **Reading in Data to SQLite using the `cursor` Object**

In [ ]:
conn2 = sqlite3.connect('example2.db')

In [ ]:
cursor = conn2.cursor()

In [ ]:
#create new SQLite table with cursor
cursor.execute("""
CREATE TABLE IF NOT EXISTS household_power (
    Date TEXT,
    Time TEXT,
    global_active_power REAL,
    global_reactive_power REAL,
    Voltage REAL,
    global_intensity REAL,
    sub_metering_1 REAL,
    sub_metering_2 REAL,
    sub_metering_3 REAL
);
""")

#commit changes
conn2.commit()

In [ ]:
#open text file and read all rows
with open(raw_data_path, 'r') as file:
    reader = csv.reader(file, delimiter=';')

    #skip header row
    header = next(reader)

    #replace missing values with "None" and prepare the data
    data = [
        [None if value == '?' else value for value in row]
        for row in reader
    ]

#use "executemany" to insert all rows at once
insert_query = """
INSERT INTO household_power (
    Date, Time, global_active_power, global_reactive_power,
    Voltage, global_intensity, sub_metering_1, sub_metering_2, sub_metering_3
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
"""
cursor.executemany(insert_query, data)

#commit changes
conn2.commit()

## **Running Queries on the SQLite Database with the `cursor` Object**

In [ ]:
#query database to retrieve column names
query = "PRAGMA table_info(household_power);"
cursor = conn2.cursor()
cursor.execute(query)
result = cursor.fetchall()

#print metadata for all columns
for column in result:
    print(column)

In [ ]:
#find number of household where global_active_power exceeds 5
query = """
SELECT COUNT(*) AS count_above_threshold
FROM household_power
WHERE global_active_power > 5.0;
"""

cursor = conn2.cursor()
cursor.execute(query)
result = cursor.fetchone()

print(result)

In [ ]:
#finds the 5 times of day with the highest total power usage across all days in the dataset
query = """
SELECT Time, SUM(global_active_power) AS total_power
FROM household_power
GROUP BY Time
ORDER BY total_power DESC
LIMIT 5;
"""

cursor = conn2.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

In [ ]:
#close connection after finishing queries
conn2.close()

# **References and Additional Learning**

## **Data**

- **[Individual Household Electric Power Consumption](https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption) from UC Irvine's Machine Learning Repository**

## **Documentation**

- **[SQLite Documentation](https://www.sqlite.org/) from sqlite.org**

# **Connect**
- **Feel free to connect with Adrian on [YouTube](https://www.youtube.com/channel/UCPuDxI3xb_ryUUMfkm0jsRA), [LinkedIn](https://www.linkedin.com/in/adrian-dolinay-frm-96a289106/), [X](https://twitter.com/DolinayG), [GitHub](https://github.com/ad17171717), [Medium](https://adriandolinay.medium.com/) and [Odysee](https://odysee.com/@adriandolinay:0). Happy coding!**

# **Podcast**

- **Check out Adrian's Podcast, The Aspiring STEM Geek on [YouTube](https://www.youtube.com/@AdrianDolinay/podcasts), [Spotify](https://open.spotify.com/show/60dPNJbDPaPw7ru8g5btxV), [Apple Podcasts](https://podcasts.apple.com/us/podcast/the-aspiring-stem-geek/id1765996824), [Audible](https://www.audible.com/podcast/The-Aspiring-STEM-Geek/B0DC73S9SN?eac_link=MCFKvkxuqKYU&ref=web_search_eac_asin_1&eac_selected_type=asin&eac_selected=B0DC73S9SN&qid=IrZ84nGqvz&eac_id=141-8769271-5781515_IrZ84nGqvz&sr=1-1) and [iHeart Radio](https://www.iheart.com/podcast/269-the-aspiring-stem-geek-202676097/)!**